# Working with Dates Basics
[xxx](https://blogs.sas.com/content/sgf/2023/04/19/getting-started-with-python-integration-to-sas-viya-part-18-update-rows-in-a-table/)

In [114]:
## Packages
import swat
import os
import pandas as pd
pd.set_option('display.max_columns', 50)
import numpy as np


## custom personal module to connect to my CAS environment
try:
    from casConnect import connect_to_cas 
except:
    print('CasConnect package not available')

## Make a Connection to CAS (REQUIRED: MODIFY CONNECTION INFORMATION)

##### To connect to the CAS server you will need:
1. the host name, 
2. the portnumber, 
3. your user name, and your password.

Visit the documentation [Getting Started with SAS® Viya® for Python](https://go.documentation.sas.com/doc/en/pgmsascdc/default/caspg3/titlepage.htm) for more information about connecting to CAS.

**Be aware that connecting to the CAS server can be implemented in various ways, so you might need to see your system administrator about how to make a connection. Please follow company policy regarding authentication.**

In [116]:
##
## Connect to CAS
##

## General connection syntax
# conn = swat.CAS(host, port, username, password)

## SAS Viya for Learners 3.5 connection
# hostValue = os.environ.get('CASHOST')
# portValue = os.environ.get('CASPORT')
# passwordToken=os.environ.get('SAS_VIYA_TOKEN')
# conn = swat.CAS(hostname=hostValue, port=portValue, password=passwordToken)

## Personal connection
try:
    conn = connect_to_cas()
    print('CAS connection succesful')
    print(conn)
except:
    print('No connection')
    pass

CAS connection succesful
CAS('ssemonthly.demo.sas.com', 443, protocol='https', name='py-session-2', session='05171c88-bc7b-9246-b144-5c00462d03b3')


## Enter your connection information to CAS below

In [117]:
## conn = swat.CAS()

## Create demo CAS table

In [135]:
## Create demo DataFrame
df = pd.DataFrame({
    'SASDate':[18260,18212, 18290, 18201, 18158],
    'StrDate':['2009-12-29', '2009-11-11', '2010-01-28', '2009-10-31', '2009-09-18']
})

## Upload DataFrame to CAS
castbl = conn.upload_frame(df, 
                           casout = {'name':'date_table', 
                                     'caslib':'casuser', 
                                     'replace':True})

NOTE: Cloud Analytic Services made the uploaded file available as table DATE_TABLE in caslib CASUSER(Peter.Styliadis@sas.com).
NOTE: The table DATE_TABLE has been created in caslib CASUSER(Peter.Styliadis@sas.com) from binary data uploaded to Cloud Analytic Services.


In [136]:
castbl.params

{'name': 'DATE_TABLE', 'caslib': 'CASUSER(Peter.Styliadis@sas.com)'}

In [137]:
castbl.head()

,SASDate,StrDate
0,18260.0,2009-12-29
1,18212.0,2009-11-11
2,18290.0,2010-01-28
3,18201.0,2009-10-31
4,18158.0,2009-09-18


In [131]:
castbl.eval('NewCol = badfunction(SASData)')

ERROR: The user-defined programming statements could not be parsed.
ERROR: Function BADFUNCTION not found.
ERROR: The user-defined programming statements could not be parsed.
ERROR: The action stopped due to errors.


SWATError: The user-defined programming statements could not be parsed.

In [132]:
castbl.params

{'name': 'DATE_TABLE',
 'caslib': 'samples',
 'computedvars': ['NewCol'],
 'computedvarsprogram': 'NewCol =  badfunction(SASData); NewCol = NewCol; '}

View the CAS table.

In [121]:
castbl.head()

,SASDate,StrDate
0,18260.0,2009-12-29
1,18212.0,2009-11-11
2,18290.0,2010-01-28
3,18201.0,2009-10-31
4,18158.0,2009-09-18


View column metadata of the CAS table.

In [95]:
castbl.columnInfo()

,Column,Label,ID,Type,RawLength,FormattedLength,Format,NFL,NFD
0,SASDate,,1,double,8,12,,0,0
1,StrDate,,2,varchar,10,10,,0,0


## Adding a format to a SAS date value

Add a SAS format to a SAS data value. SAS stores dates as the number of days since January 1, 1960. A SAS format displays the date in a human readable format.

In [122]:
castbl.alterTable(columns = [
        {'name':'SASDate', 'format':'mmddyy10.'}
])
castbl.head()

,SASDate,StrDate
0,2009-12-29,2009-12-29
1,2009-11-11,2009-11-11
2,2010-01-28,2010-01-28
3,2009-10-31,2009-10-31
4,2009-09-18,2009-09-18


View the column metadata. Notice the SASData column is still a double, but now it contains a format.

In [123]:
castbl.columnInfo()

,Column,Label,ID,Type,RawLength,FormattedLength,Format,NFL,NFD
0,SASDate,,1,double,8,12,MMDDYY,10,0
1,StrDate,,2,varchar,10,10,,0,0


## Converting a date string to SAS date value
The StrDate column is simply a string column. We need to convert the string to a SAS date value. The [INPUT function](https://documentation.sas.com/doc/en/pgmsascdc/v_038/lefunctionsref/p19en16vskd2vhn1vwmxpxnglxxs.htm) converts strings to SAS numeric values.

In [124]:
castbl.eval('NewDate = input(StrDate,yymmdd10.)')

In [125]:
castbl.head()

,SASDate,StrDate,NewDate
0,2009-12-29,2009-12-29,18260.0
1,2009-11-11,2009-11-11,18212.0
2,2010-01-28,2010-01-28,18290.0
3,2009-10-31,2009-10-31,18201.0
4,2009-09-18,2009-09-18,18158.0


## Using the SWAT package datetime properties
CASColumn.dt can be used to access the values of a CAS table column as datetime-like properties. They are accessed as CASColumn.dt.<property>.
    
[Documentation](https://sassoftware.github.io/python-swat/api.html#datetime-properties)

In [100]:
castbl['Year_dt'] = castbl.SASDate.dt.year
castbl.head()

,SASDate,StrDate,NewDate,Year_dt
0,2009-12-29,2009-12-29,18260.0,2009.0
1,2009-11-11,2009-11-11,18212.0,2009.0
2,2010-01-28,2010-01-28,18290.0,2010.0
3,2009-10-31,2009-10-31,18201.0,2009.0
4,2009-09-18,2009-09-18,18158.0,2009.0


In [101]:
castbl['Month_dt'] = castbl.SASDate.dt.month
castbl.head()

,SASDate,StrDate,NewDate,Year_dt,Month_dt
0,2009-12-29,2009-12-29,18260.0,2009.0,12.0
1,2009-11-11,2009-11-11,18212.0,2009.0,11.0
2,2010-01-28,2010-01-28,18290.0,2010.0,1.0
3,2009-10-31,2009-10-31,18201.0,2009.0,10.0
4,2009-09-18,2009-09-18,18158.0,2009.0,9.0


The day of the week (Monday=0, Sunday=6)

In [102]:
castbl['DayOfWeek_dt'] = castbl.SASDate.dt.dayofweek
castbl.head()

,SASDate,StrDate,NewDate,Year_dt,Month_dt,DayOfWeek_dt
0,2009-12-29,2009-12-29,18260.0,2009.0,12.0,1.0
1,2009-11-11,2009-11-11,18212.0,2009.0,11.0,2.0
2,2010-01-28,2010-01-28,18290.0,2010.0,1.0,3.0
3,2009-10-31,2009-10-31,18201.0,2009.0,10.0,5.0
4,2009-09-18,2009-09-18,18158.0,2009.0,9.0,4.0


In [103]:
castbl['DayOfWeek_dt'] = castbl.NewDate.dt.dayofweek
castbl.head()

TypeError: double columns must have a date, time, or datetime format

## Using SAS functions

In [104]:
castbl.eval('Year_sas_func = year(NewDate)')
castbl.head()

,SASDate,StrDate,NewDate,Year_dt,Month_dt,DayOfWeek_dt,Year_sas_func
0,2009-12-29,2009-12-29,18260.0,2009.0,12.0,1.0,2009.0
1,2009-11-11,2009-11-11,18212.0,2009.0,11.0,2.0,2009.0
2,2010-01-28,2010-01-28,18290.0,2010.0,1.0,3.0,2010.0
3,2009-10-31,2009-10-31,18201.0,2009.0,10.0,5.0,2009.0
4,2009-09-18,2009-09-18,18158.0,2009.0,9.0,4.0,2009.0


In [105]:
castbl.eval('Month_sas_func = month(SASDate)')
castbl.head()

,SASDate,StrDate,NewDate,Year_dt,Month_dt,DayOfWeek_dt,Year_sas_func,Month_sas_func
0,2009-12-29,2009-12-29,18260.0,2009.0,12.0,1.0,2009.0,12.0
1,2009-11-11,2009-11-11,18212.0,2009.0,11.0,2.0,2009.0,11.0
2,2010-01-28,2010-01-28,18290.0,2010.0,1.0,3.0,2010.0,1.0
3,2009-10-31,2009-10-31,18201.0,2009.0,10.0,5.0,2009.0,10.0
4,2009-09-18,2009-09-18,18158.0,2009.0,9.0,4.0,2009.0,9.0


The WEEKDAY function produces an integer that represents the day of the week, where 1=Sunday, 2=Monday, ..., 7=Saturday.

In [107]:
castbl.eval('DayOfWeek_sas_func = weekday(SASDate)')
castbl.head()

,SASDate,StrDate,NewDate,Year_dt,Month_dt,DayOfWeek_dt,Year_sas_func,Month_sas_func,DayOfWeek_sas_func
0,2009-12-29,2009-12-29,18260.0,2009.0,12.0,1.0,2009.0,12.0,3.0
1,2009-11-11,2009-11-11,18212.0,2009.0,11.0,2.0,2009.0,11.0,4.0
2,2010-01-28,2010-01-28,18290.0,2010.0,1.0,3.0,2010.0,1.0,5.0
3,2009-10-31,2009-10-31,18201.0,2009.0,10.0,5.0,2009.0,10.0,7.0
4,2009-09-18,2009-09-18,18158.0,2009.0,9.0,4.0,2009.0,9.0,6.0


## CAS tables are lazily evaluated

In [113]:
castbl.copyTable(casout = {'name':'NewTable', 'caslib':'casuser', 'replace':True})
conn.tableInfo(caslib = 'casuser')

SWATError: HTTP/1.1 401 Unauthorized

## Terminate the CAS connection

In [16]:
conn.terminate()